In [2]:
import pandas as pd
import numpy as np
import yaml
import os
from geopy.geocoders import Nominatim
from tqdm import tqdm
from glob import glob

In [ ]:
## matchid,venue,country,date,month,year,match_between,toss_winner,toss_decision,
##current_innings,over,batsman,non_striker,bowler,extras,total_runs,batsman_runs,extras_type
##out,out_batsman,dismissal_type,dismissed_by_fielder,
##winner,win_by,method_if_any,super_over,super_over_winner,potm

df_data = []

def get_country(venue):
    
    geolocator = Nominatim(user_agent="geoapiExercises")
    location = geolocator.geocode(venue,language="en")
    country = location.address.split(",")[-1].strip()
    return country


def create_csv(folder_path):
    
   
    for file in tqdm(os.listdir(folder_path)):
        #print(file)
        try:
            with open(folder_path+file) as stream:
                data = yaml.safe_load(stream)

                ####### Static Data#####
                match_id = file.split(".")[0]
                try:
                    venue_city = data["info"]["city"]
                except:    
                    venue_city = data["info"]["venue"]
                #print(venue_city)
                try:
                    country = get_country(venue_city)
                except:
                    country = "None"
                print(file,venue_city,country)
                #print(data["info"]["dates"][0],str(data["info"]["dates"][0]).split("-"))
                day,month,year = str(data["info"]["dates"][0]).split("-")
                if "eliminator" in list(data["info"]["outcome"].keys()):
                    #super_over = "Yes"
                    super_over_winner_team = data["info"]["outcome"]["eliminator"]
                    winner_team = "Tie"
                    winner_method = "Tie"
                    winner_num = "Tie"

                elif "result" in list(data["info"]["outcome"].keys()):
                    if data["info"]["outcome"] == "no result":

                        winner_team = "No Result"
                        winner_method = "No Result"
                        winner_num = "No Result"
                        super_over_winner_team = "None"

                else:
                    winner_team = data["info"]["outcome"]["winner"]
                    if "by" in list(data["info"]["outcome"].keys()):
                        winner_method = list(data["info"]["outcome"]["by"].keys())[0]
                        winner_num = list(data["info"]["outcome"]["by"].values())[0]
                    else:
                        ###Awarded Methods###
                        winner_method = data["info"]["outcome"]["method"]
                        winner_num = "None"
                    #super_over = "No"
                    super_over_winner_team = "None"

                match_between = data["info"]["teams"][0] + " vs " + data["info"]["teams"][1]
                toss_winner = data["info"]["toss"]["winner"]
                toss_decision = data["info"]["toss"]["decision"]
                try:
                    potm = "/".join((data["info"]["player_of_match"]))
                except:
                    potm = "No Data"

                if "method" in data["info"]["outcome"].keys():
                    method_if_any = data["info"]["outcome"]["method"]
                else:
                    method_if_any = "None"
                #############################


                for inn in data["innings"]:
                    innings = list(inn.values())[0]
                    #print(innings)
                    current_innings_team = innings["team"]
                    if "super over" in list(inn.keys())[0].lower():
                        super_over = "Yes"
                    else:
                        super_over = "No"
                    for delivery in list(inn.values())[0]["deliveries"]:
                        #print(delivery)
                        over_number = list(delivery.keys())[0]
                        over_values = list(delivery.values())[0]
                        batsman = over_values["batsman"]
                        non_striker = over_values["non_striker"]
                        bowler = over_values["bowler"]
                        extras,total_runs,batsman_runs = over_values["runs"]["extras"],over_values["runs"]["total"],over_values["runs"]["batsman"]
                        if extras > 0:
                            extras_type = list(over_values["extras"].keys())[0]
                        else:
                            extras_type = "None"

                        if "wicket" in over_values.keys():
                            out = "Yes"
                            player_out = over_values["wicket"]["player_out"]
                            out_type = over_values["wicket"]["kind"]
                            if "fielders" in over_values["wicket"]:
                                dismissed_by_fielder = "/".join((over_values["wicket"]["fielders"]))
                            else:
                                dismissed_by_fielder = "None"
                        else:
                            out = "No"
                            player_out = "None"
                            out_type = "None"
                            dismissed_by_fielder = "None"

                        df_data.append([match_id,venue_city,country,day,month,year,match_between,toss_winner,toss_decision,\
                       current_innings_team,over_number,batsman,non_striker,bowler,extras,total_runs,batsman_runs,\
                                    extras_type,\
                        out,player_out,out_type,dismissed_by_fielder,winner_team,winner_method,winner_num,method_if_any,\
                                        super_over,super_over_winner_team,potm])


        #return df_data
        except TypeError as e:
            print("passed")
            pass

In [ ]:
create_csv("/home/shivargha/performance_prediction/T20_cricket/bbl/")
df = pd.DataFrame(df_data,
                  columns=["matchid","venue","country","date","month","year","match_between",
                           "toss_winner","toss_decision",\
                          "current_innings","over","batsman","non_striker","bowler","extras",\
                           "total_runs","batsman_runs",\
                          "extras_type","out","out_batsman","dismissal_type","dismissed_by_fielder",\
                          "winner","winner_method","win_by","method_if_any","super_over","super_over_winner","potm" 
                          ])

In [ ]:
df.to_csv("/home/shivargha/performance_prediction/T20_cricket/bbl.csv")

In [3]:
############## MY analysis ##############
df_t20 = pd.read_csv("T20_cricket/t20i.csv")

/home/shivargha/anaconda3/envs/env_py37/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3553: DtypeWarning: Columns (25) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [8]:
df_t20[(df_t20["country"] == "New Zealand")]

,Unnamed: 0,matchid,venue,country,date,month,year,match_between,toss_winner,toss_decision,...,out_batsman,dismissal_type,dismissed_by_fielder,winner,winner_method,win_by,method_if_any,super_over,super_over_winner,potm
1179,1179,473920,Christchurch,New Zealand,2010,12,30,New Zealand vs Pakistan,Pakistan,bat,...,None,None,None,Pakistan,runs,103,None,No,None,Abdul Razzaq
1180,1180,473920,Christchurch,New Zealand,2010,12,30,New Zealand vs Pakistan,Pakistan,bat,...,None,None,None,Pakistan,runs,103,None,No,None,Abdul Razzaq
1181,1181,473920,Christchurch,New Zealand,2010,12,30,New Zealand vs Pakistan,Pakistan,bat,...,None,None,None,Pakistan,runs,103,None,No,None,Abdul Razzaq
1182,1182,473920,Christchurch,New Zealand,2010,12,30,New Zealand vs Pakistan,Pakistan,bat,...,None,None,None,Pakistan,runs,103,None,No,None,Abdul Razzaq
1183,1183,473920,Christchurch,New Zealand,2010,12,30,New Zealand vs Pakistan,Pakistan,bat,...,None,None,None,Pakistan,runs,103,None,No,None,Abdul Razzaq
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
376818,376818,386494,Christchurch,New Zealand,2009,2,25,New Zealand vs India,New Zealand,field,...,None,None,None,New Zealand,wickets,7,None,No,None,BB McCullum
376819,376819,386494,Christchurch,New Zealand,2009,2,25,New Zealand vs India,New Zealand,field,...,None,None,None,New Zealand,wickets,7,None,No,None,BB McCullum
376820,376820,386494,Christchurch,New Zealand,2009,2,25,New Zealand vs India,New Zealand,field,...,None,None,None,New Zealand,wickets,7,None,No,None,BB McCullum
376821,376821,386494,Christchurch,New Zealand,2009,2,25,New Zealand vs India,New Zealand,field,...,None,None,None,New Zealand,wickets,7,None,No,None,BB McCullum


In [10]:
df_nzind = df_t20[df_t20["match_between"] == "New Zealand vs India"]

In [25]:
df_nzind.columns

Index(['Unnamed: 0', 'matchid', 'venue', 'country', 'date', 'month', 'year',
       'match_between', 'toss_winner', 'toss_decision', 'current_innings',
       'over', 'batsman', 'non_striker', 'bowler', 'extras', 'total_runs',
       'batsman_runs', 'extras_type', 'out', 'out_batsman', 'dismissal_type',
       'dismissed_by_fielder', 'winner', 'winner_method', 'win_by',
       'method_if_any', 'super_over', 'super_over_winner', 'potm'],
      dtype='object')

In [28]:
df_nzind["batsman"].unique()

array(['KL Rahul', 'SV Samson', 'RG Sharma', 'SS Iyer', 'S Dube',
       'MK Pandey', 'MJ Guptill', 'C Munro', 'TL Seifert', 'TC Bruce',
       'LRPL Taylor', 'DJ Mitchell', 'MJ Santner', 'SC Kuggeleijn',
       'IS Sodhi', 'TG Southee', 'HK Bennett', 'MS Chapman',
       'GD Phillips', 'R Ravindra', 'SA Yadav', 'RR Pant', 'VR Iyer',
       'AR Patel', 'KS Williamson', 'C de Grandhomme', 'S Dhawan',
       'V Shankar', 'HH Pandya', 'MS Dhoni', 'KD Karthik', 'KH Pandya',
       'B Kumar', 'YS Chahal', 'KK Ahmed', 'JDS Neesham', 'AF Milne',
       'V Kohli', 'Washington Sundar', 'SN Thakur', 'NA Saini',
       'G Gambhir', 'V Sehwag', 'SK Raina', 'Yuvraj Singh', 'YK Pathan',
       'RA Jadeja', 'IK Pathan', 'JD Ryder', 'BB McCullum', 'JDP Oram',
       'NT Broom', 'NL McCullum', 'JJ Bumrah', 'Harbhajan Singh'],
      dtype=object)

In [33]:
df_nzind[df_nzind["batsman"] == "RR Pant"]["year"].unique()

array([17, 10,  6, 19,  8])

In [31]:
sum(df_nzind[df_nzind["batsman"] == "RR Pant"]["batsman_runs"])

101